In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline


Advanced time-domain photonic circuits
======================================

*Authors: Fabian Laudenbach and Theodor Isacsson*

In the `beginner's demo <link-to-beginner's-demo>`_ we explained how Xanadu's
quantum-advantage machine *Borealis* can be programmed in a fairly
straightforward way, using some convenience functions that helped us make our
user-defined circuit compatible with the hardware. In this advanced tutorial, we
will again define and submit an instance of Gaussian Boson Sampling (GBS), but
this time we will carry out the steps more manually. This allows us to
understand better the subtleties of time-domain programs and exercise more
control over the quantum circuit we create.

Let's have a look at the schematic below. It looks very much like the one shown
in the beginner's demo, only it's slightly more accurate as we have added two
more details:

- There is now a phase gate located in each of the three delay lines.
- After the three cascaded loops, the one spatial mode branches out into a
  binary tree of 16 spatial modes, each one probed by its own photon-number
  resolving detector.

![](/tutorials/images/borealis_schematic_advanced.png)

   :width: 100%
   :align: center

This advanced tutorial will direct more focus on these aspects and help us to
get a more comprehensive understanding of *Borealis*.


Loading the device specification
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

We again need to import Strawberry Fields along with NumPy and load the
*Borealis* ``device`` object.




In [ ]:
import strawberryfields as sf
import numpy as np

eng = sf.RemoteEngine("borealis")
device = eng.device

<div class="alert alert-info"><h4>Note</h4><p>If you don't have an API key, you can simply download a dummy device
    specification and hardware certificate from `here <link-to-dummy-device-spec>`__.
    Load these files and use them to create a ``device`` object like what is shown
    below. This object will allow you to go through most steps of this tutorial,
    although you won't be able to run anything on the *Borealis* hardware
    device, nor receive any samples.

    .. code::

        import json

        device_spec = json.load(open("device_specification.json"))
        device_cert = json.load(open("device_certificate.json"))
        device = sf.Device(spec=device_spec, cert=device_cert)</p></div>




Defining a GBS circuit
~~~~~~~~~~~~~~~~~~~~~~

As in the beginner's demo, we will define a GBS circuit that we can later
submit to the hardware and/or to local simulations. Only this time, we want to
do it more manually, relying less on convenience functions such as
``strawberryfields.tdm.borealis_gbs`` or
``strawberryfields.tdm.borealis_full_compile``.

We want to run a GBS instance similar to the one in the beginner's tutorial.
One of the things that the helper functions in the be beginner's demo did for
us was to match whatever squeezing value we have defined to the closest one
supported by the hardware. Now, we will define a supported squeezing value
right away using the device certificate.



In [ ]:
modes = 216

r0 = device.certificate["squeezing_parameters_mean"]["high"]
r = [r0] * modes

Next, we will define our GBS circuit by randomizing the phase-gate and
beamsplitter arguments, very much like we did in the beginner's demo. Also, we
will make sure again that the loops are completely filled with computational
modes at the beginning of the circuit. So we will open the loops in the
initial time bins by setting them to $T=1$ ($\alpha=0$).



In [ ]:
min_phi, max_phi = -np.pi / 2, np.pi / 2
min_T, max_T = 0.4, 0.6

# rotation-gate parameters
phi_0 = np.random.uniform(low=min_phi, high=max_phi, size=modes).tolist()
phi_1 = np.random.uniform(low=min_phi, high=max_phi, size=modes).tolist()
phi_2 = np.random.uniform(low=min_phi, high=max_phi, size=modes).tolist()

# beamsplitter parameters
T_0 = np.random.uniform(low=min_T, high=max_T, size=modes)
T_1 = np.random.uniform(low=min_T, high=max_T, size=modes)
T_2 = np.random.uniform(low=min_T, high=max_T, size=modes)
alpha_0 = np.arccos(np.sqrt(T_0)).tolist()
alpha_1 = np.arccos(np.sqrt(T_1)).tolist()
alpha_2 = np.arccos(np.sqrt(T_2)).tolist()

# the travel time per delay line in time bins
delay_0, delay_1, delay_2 = 1, 6, 36

# set the first beamsplitter arguments to ``T=1`` (``alpha=0``) to fill the
# loops with pulses
alpha_0[:delay_0] = [0.0] * delay_0
alpha_1[:delay_1] = [0.0] * delay_1
alpha_2[:delay_2] = [0.0] * delay_2

Now here comes a tricky part. Although these gate sequences are of length 216
(the number of squeezed-light modes in our GBS instance) the actual
time-domain program needs to be longer than that. Why is that the case?

We just imposed that the first computational mode is fully coupled into the
first loop. This means that it will arrive at the second loop after exactly
one time bin (i.e. one roundtrip in the first delay line). We also imposed
that the first mode should be completely coupled into the second loop as well,
which means it will arrive at the third loop no earlier 7 time bins after the
program has started (1 + 6 time bins delay). Therefore, the gates associated
to the second and third loop need some idle time bins, awaiting the first
computational mode.

We thus need to append 0-valued parameters (i.e. full transmission) to the
gates in the first two loops. This will make sure that the non-trivial random
gates are applied to the computational modes exclusively.




In [ ]:
# second loop needs to await the first mode for 1 time bin (delay_0 = 1)
phi_1 = [0] * delay_0 + phi_1
alpha_1 = [0] * delay_0 + alpha_1

# second loop needs to await the first mode for 7 time bins (delay_0 + delay_1 = 1 + 6)
phi_2 = [0] * (delay_0 + delay_1) + phi_2
alpha_2 = [0] * (delay_0 + delay_1) + alpha_2

Toward the end of the time-domain program, we need to make sure that all
computational modes in the loops are entirely coupled out again, avoiding
interfering with the vacuum modes that will follow the pulse train of 216
squeezed-light modes. This means we need to set the beamsplitters to full
transmission again, and, on top of this, since all gate sequences should be of
same length, the gates of the first and second loop need to remain idle until
the very last computational mode has exited the third loop.

This requires us to add 0-valued parameters corresponding to the number of
modes required to exit through the current, and all coming, loop(s); i.e., 43
(1 + 6 + 36) for the first loop, 42 (6 + 36) for the second loop, and 36 for
the third loop.




In [ ]:
# empty first loop (delay_0) and await the emptying of second and third loop
# (delay_1 + delay_2)
r += [0] * (delay_0 + delay_1 + delay_2)
phi_0 += [0] * (delay_0 + delay_1 + delay_2)
alpha_0 += [0] * (delay_0 + delay_1 + delay_2)

# empty second loop (delay_1) and await the emptying of third loop (delay_2)
phi_1 += [0] * (delay_1 + delay_2)
alpha_1 += [0] * (delay_1 + delay_2)

# empty third loop
phi_2 += [0] * delay_2
alpha_2 += [0] * delay_2

![](/tutorials/images/borealis_vacuum_padding.png)

   :width: 100%
   :align: center
   :target: javascript:void(0);




Unfortunately, all this padding with idle time bins before and after the
random gates is necessary to avoid interference of computational modes with
vacuum. This simply reflects the reality that we need a total unitary that
fits the size of the total input state: 216 computational modes and 43 vacuum
modes (1, 6, 36 of them in the first, second and third loop, respectively).
Our total unitary thus needs to be of size 259 $\times$ 259. Check out
the image above for an overview of the necessary vacuum mode padding.

However, the reward of all precautions taken above is that the unitary that we
*care* about is now a block matrix within the total unitary, acting on the 216
computational modes exclusively. Moreover, the length of the argument lists
submitted to the six quantum gates is now exactly 259, matching the size of
the total unitary. Let's collect all the gate sequences in one list that we
can submit to the ``sf.TDMProgram``. The list, aptly named ``gate_args_list``,
contains all gate parameters in the order in which the corresponding gates
will be applied.



In [ ]:
gate_args_list = [r, phi_0, alpha_0, phi_1, alpha_1, phi_2, alpha_2]

[len(gate_args) for gate_args in (gate_args_list)]

.. rst-class:: sphx-glr-script-out

 Out:

 .. code-block:: none

   [259, 259, 259, 259, 259, 259, 259]




As mentioned in the beginner's demo, each loop applies a static phase rotation
to the modes in it. Currently, this phase rotation cannot be programmed. All
we can do is to *measure* this phase and take it into account whenever we run
a program on the machine. This is why the three loop offsets ``"loop_phases"``
are part of the device certificate. Even though we have no direct influence on
them, we need to deal with them in order to obtain a correct representation of
the circuit that is applied. This can be done in two different ways:

-  We can stay agnostic about the static offsets and use our phase modulators
   to actively compensate for them in the background. This is done
   automatically when the loop offsets do not occur in the ``TDMProgram``
   submitted to the hardware. In this case, the Strawberry Fields compiler
   will absorb the three roundtrip phases into the instructions for the phase
   modulators such that they *effectively* apply the user-defined phase-gate
   arguments.
-  We include these offsets explicitly into our ``TDMProgram``. In this case,
   no hidden compensation takes place and the modulators apply exactly the
   phases defined by the user.

While in the beginner's demo we went for the first option, we will go with the
second option here. In other words, we will acknowledge the roundtrip phases
as part of our circuit and will explicitly add them to our time-domain
program. In order to do so, let's obtain them from the certificate.



In [ ]:
# the intrinsic roundtrip phase of each delay line
phis_loop = device.certificate["loop_phases"]

Excecuting the circuit
 ~~~~~~~~~~~~~~~~~~~~~~

######################################################################
 Submit to hardware
 <<<<<<<<<<<<<<<<<<

 From here on, we can proceed very much like we did in the beginner's demo.
 Only this time, our ``TDMProgram`` includes explicit mentions of the
 loop-roundtrip phases.



In [ ]:
from strawberryfields.tdm import get_mode_indices
from strawberryfields.ops import Sgate, Rgate, BSgate, MeasureFock

delays = [1, 6, 36]
vac_modes = sum(delays)
n, N = get_mode_indices(delays)

prog = sf.TDMProgram(N)
with prog.context(*gate_args_list) as (p, q):
    Sgate(p[0]) | q[n[0]]

    for i in range(len(delays)):
        Rgate(p[2 * i + 1]) | q[n[i]]
        BSgate(p[2 * i + 2], np.pi / 2) | (q[n[i + 1]], q[n[i]])
        Rgate(phis_loop[i]) | q[n[i]]

    MeasureFock() | q[0]

shots = 250_000

eng = sf.RemoteEngine("borealis")
results = eng.run(prog, shots=shots)

Running the circuit remotely on *Borealis* will return a ``results`` object,
which contains the samples that are obtained by the machine. The samples
returned by any ``sf.TDMProgram`` are of shape ``(shots, spatial_modes,
temporal_modes)``, so in our case whe should have gotten ``(shots, 1, 259)``.
But since the first 43 modes that arrived at the detectors are the vacuum
modes that had lived in the loop before the program started, let's crop the
samples array and only focus on the 216 *computational* modes. As seen in the
beginner's demo, the same thing can be achieved by passing ``crop=True`` to
the ``eng.run()`` call.



In [ ]:
samples = results.samples[..., 43:]

Submit to realistic simulation <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

Now, let us switch to a local simulation. In order to make it realistic, we
want to take optical loss into account. In the beginner's demo, realistic loss
was added by the compiler in the backend. Here, we want to insert the various
loss components manually into our circuit. The daily calibration routine run
on *Borealis* decomposes the total loss suffered by our pulses into three
categories:

- ``eta_glob``: The total loss experienced by the different modes depends on
  which loops they have travelled through and on which detector they ended up
  in. So it will be highly inhomogeneous within the 216 computational modes.
  ``eta_glob``, on the other hand, describes their *common* efficiency. It is
  experienced by a mode that bypasses all three loops and ends up in our most
  efficient detector. In other words ``1 - eta_glob`` is the minimal loss that
  every mode will suffer. On top of that, most modes will suffer additional
  loss imposed by the loops and a less efficient detector.
- ``eta_loop``: This is simply a list containing the respective efficiencies
  of the three loop.
- ``eta_ch_rel``: In the experiment, two consecutive pulses are only 167 ns
  apart which exceeds the rate that one single photon-number resolving (PNR)
  detector can handle. So although our batches of 216 squeezed-light modes
  travel towards the detection module in one single fibre, they need to be
  demultiplexed into 16 spatial modes, each one of them probed by its own PNR
  detector. (You can find a schematic of our demultiplexing-and-detection
  module illustrated in the top figure of this tutorial.) Now, instead of one
  PNR operating at 6 MHz, we have 16 PNRs operating at 6/16 MHz. The
  demultiplexing pattern is repetitive, meaning that detector $i$ will
  detect all temporal modes whose index satisfies $i + 16n$ where
  $n$ is an integer. Since the 16 spatial modes and their respective
  detectors do not share the exact same efficiency, ``eta_ch_rel`` keeps track
  of the loss suffered by each one of our modes individually.

 All of these loss components can be obtained from the ``device`` object.



In [ ]:
eta_glob = device.certificate["common_efficiency"]
etas_loop = device.certificate["loop_efficiencies"]
etas_ch_rel = device.certificate["relative_channel_efficiencies"]

Note that the detector-channel efficiencies are cyclic, so mode ``i`` sees the
same efficiency as mode ``i + 16``, thus necessitating us to repeat the 16
efficiency values for the length of the program. We will also add the channel
efficiencies to our list of gate arguments that we submit to the
``TDMProgram``. This is because the relative detection efficiency experienced
by the modes needs to be updated at each time bin and is therefore a dynamic
TDM gate, very much like the phase modulators and beamsplitters.



In [ ]:
prog_length = len(gate_args_list[0])
reps = int(np.ceil(prog_length / 16))
etas_ch_rel = np.tile(etas_ch_rel, reps)[:prog_length]
etas_ch_rel = etas_ch_rel.tolist()

gate_args_list += [etas_ch_rel]

With that, we are now able to redefine the circuit that we submitted to
*Borealis* for simulations under realistic loss.



In [ ]:
from strawberryfields.ops import LossChannel

prog_sim = sf.TDMProgram(N)
with prog_sim.context(*gate_args_list) as (p, q):
    Sgate(p[0]) | q[n[0]]
    LossChannel(eta_glob) | q[n[0]]

    for i in range(len(delays)):
        Rgate(p[2 * i + 1]) | q[n[i]]
        BSgate(p[2 * i + 2], np.pi / 2) | (q[n[i + 1]], q[n[i]])
        Rgate(phis_loop[i]) | q[n[i]]
        LossChannel(etas_loop[i]) | q[n[i]]

    LossChannel(p[7]) | q[0]
    MeasureFock() | q[0]

As we have now inserted loss explicitly to the circuit, we can skip any
*Borealis*-specific compiling steps and submit our program directly to the
Gaussian backend.



In [ ]:
eng_sim = sf.Engine(backend="gaussian")
results_sim = eng_sim.run(prog_sim, shots=None, space_unroll=True)

The covariance matrix returned by ``results_sim.state.cov()`` describes our 216
computational modes as well as the 43 vacuum modes that were stored in the
loops before execution of the program. In the beginner's demo we made sure
that only the computational modes are considered in the covariance matrix by
adding ``crop=True`` to the ``run()`` method. We could have done the same
thing here, but instead let's crop out the vacuum modes more manually using the
package *The Walrus*.



In [ ]:
from thewalrus.quantum import reduced_gaussian

# total covariance matrix and mean vector describe 259 modes
cov_tot = results_sim.state.cov()
mu_tot = np.zeros(len(cov_tot))

# cropped covariance matrix and mean vector describe 216 modes
mu, cov = reduced_gaussian(mu_tot, cov_tot, range(vac_modes, prog_length))

From here you can use the methods described in the beginner's demo to compare
your experimental results to the ones obtained by simulation.

Note that in the above ``eng_sim.run()`` call we added the flag
``space_unroll=True``. This feature was already used and very briefly
explained in the beginner's demo. Here is what this feature is about: The
circuit above will be rolled-up, using symbolic placeholder variables for the
gate parameters. The circuit must be unrolled with each mode only being
measured once, a process we've deciced to call space-unrolling.

Space-unrolling might seem slightly confusing, but it's actually fairly
straight-forward. Instead of having several modes going through the same path,
we interpret the TDM loop interactions as spatially separate. This way, each
measurement is made on a different spatial mode, as can be seen in the
right-hand side of the image below.

The left-handside depicts how an actual single loop TDM circuit would work,
while the right-hand side is how Strawberry Fields simulates it. Since each
mode can only be measured once (i.e., modes cannot be reused when using Fock
measurements, opposite to the TD2 case, which uses homodyne measurements), the
circuit needs to be unrolled. To get an understanding for how this
space-unrolling works, try to follow the interactions happening in the
left-hand side single-loop setup, and try to spot them in the right-hand side
circuit diagram. They are in practice equal!




![](/tutorials/images/borealis_space_unrolling.png)

   :width: 100%
   :align: center
   :target: javascript:void(0);




Calculating the transfer matrix
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

The circuit above can be described as a combination of initial state
preparation, followed by an interferometer over the 216 modes used in the
circuit. This interferometer setup can be described by a so-called transfer
matrix. It describes the quantum circuit applied to any initial state. In the
absence of loss, the transfer matrix is be equivalent to the unitary. By
having access to the tranfer matrix it's possible to recreate the same circuit
using different initial states. Instead of starting with a squeezed state, as
we did above, we could e.g. apply the transfer matrix to a coherent state or
GKP states.

There is a way to quickly calculate the transfer matrix of a circuit — as
long as it only contains beamsplitters, rotation gates and loss channels
— without having to execute it. This is done by using the "passive"
compiler to compile the circuit into a single transfer gate. We thus
remove the squeezing from the circuit and simply compile the program
using the "passive" compiler, extracting the parameter from the
restulting ``PassiveGate``.



In [ ]:
prog_passive = sf.TDMProgram(N)

with prog_passive.context(*gate_args_list, etas_ch_rel) as (p, q):
    LossChannel(eta_glob) | q[n[0]]

    for i in range(len(delays)):
        Rgate(p[2 * i + 1]) | q[n[i]]
        BSgate(p[2 * i + 2], np.pi / 2) | (q[n[i + 1]], q[n[i]])
        Rgate(phis_loop[i]) | q[n[i]]
        LossChannel(etas_loop[i]) | q[n[i]]

    LossChannel(p[7]) | q[0]

prog_passive.space_unroll()

prog_passive = prog_passive.compile(compiler="passive")

Keep in mind that the circuit now only consist of a single ``PassiveGate``,
which has a single matrix parameter. This parameter is the circuits transfer
matrix. The transfer matrix then allows us to experiment using the same setup,
but with different state preparations.



In [ ]:
transfer_matrix_tot = prog_passive.circuit[0].op.p[0]

# crop out the vacuum modes
transfer_matrix = transfer_matrix_tot[vac_modes:prog_length, vac_modes:prog_length]

A matrix plot will give us a good intuition on the connectivity of our
temporal modes.



In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(18, 8))
ax[0].matshow(transfer_matrix.real)
ax[1].matshow(transfer_matrix.imag)
plt.show()

![](/tutorials/images/borealis_transfer_matrix.png)

   :width: 100%
   :align: center
   :target: javascript:void(0);




Conclusion
~~~~~~~~~~

This tutorial covered how to submit jobs to our temporal-division multiplexing
device, the *Borealis*. It's rather exciting to be able to demonstrate the
concept of quantum-computational advantage by examples running on actual
hardware. As you now have seen, there are quite a few details to keep track
of, but, as long as the circuit follows the correct layout, and the parameters
are correctly declared, anyone with Xanadu cloud access can submit jobs to be
excecuted on the Borealis hardware, and get back samples that would be difficult, if
not outright impossible, to retrieve using a local simulator.

We hope that you've enjoyed this demonstration, and that you're as excited as
we are about the possibilities. If you wish to dig deeper into the science
behind all of this, we recommend you to read our paper [[#advantage2021]_].




References
~~~~~~~~~~

.. [#advantage2021]

    Quantum computational advantage with a programmable photonic processor


